<img style="float: right;" src="images/cell2mol_logo.png" width="500">

# cell2mol
Unit Cell to Molecule Interpretation

In [11]:
import numpy as np
from ipywidgets import interactive, widgets, HTML, Button
from typing import Callable
import warnings
import os
import markdown
import codecs
from IPython.display import display, FileLink
# Two imports are not needed due to visualization being disabled until nglview integration is fixed
#import ase.io
#import nglview


warnings.simplefilter('ignore')

In [12]:
html_description = markdown.markdown("""
cell2mol is able to analyze cif files containing crystallographic unit cell information and provide a chemical interpretation of the individual molecular species""")
description=HTML(html_description)

pre_accordion = widgets.Accordion()
pre_accordion.children = [description]
pre_accordion.set_title(0, 'Description')
pre_accordion.selected_index = None
pre_accordion

Accordion(children=(HTML(value='<p>cell2mol is able to analyze cif files containing crystallographic unit cell…

Proceed through the four tabs sequentially.

In [13]:
# Import the core functionalities of cell2mol
from cell2mol.cif2info import cif_2_info
from cell2mol.c2m_module import save_cell, cell2mol



In [15]:
# Define plotting and downloading functionalities

class DownloadButton(Button):
    """Download button with dynamic content

    The content is generated using a callback when the button is clicked.
    """

    def __init__(self, filename: str, contents: Callable[[], str], **kwargs):
        super(DownloadButton, self).__init__(**kwargs)
        self.filename = filename
        self.contents = contents
        self.on_click(self.__on_click)

    def __on_click(self, b):
        contents: bytes = self.contents().encode('utf-8')
        b64 = base64.b64encode(contents)
        payload = b64.decode()
        digest = hashlib.md5(contents).hexdigest()  # bypass browser cache
        id = f'dl_{digest}'

        display(HTML(f"""
<html>
<body>
<a id="{id}" download="{self.filename}" href="data:text/csv;base64,{payload}" download>
</a>

<script>
(function download() {{
document.getElementById('{id}').click();
}})()
</script>

</body>
</html>
"""))


def plot_from_file(filename):
    pass
    # This function is disabled currently due to incompatibilities depending on the nglviewer and widgets versions
    #atoms = ase.io.read(filename)
    #v = nglview.show_ase(self.conf)
    #v.add_representation("unitcell")
    #boxy=widgets.Box([v],layout=self.box_layout)
    #display(boxy)

In [34]:
#############################
# STEP 1: Choose and upload
#############################

# Define upload widget
Widget1UploadConfiguration = widgets.FileUpload(
 accept='.cif',  # Accepted file extension e.g. '.cif'
 multiple=False,  # True to accept multiple files upload else False
 wait=True
)

examples = [('cell2mol/test/cif/AJEPIP.cif')]
widget_choose_filename = widgets.Dropdown(options=examples,description='Structure:')

Widget1Out = widgets.Output()
def chooseAndPlotConfigurationAltOutput(filename):
    Widget1Out.clear_output()
    with Widget1Out:
        plot_from_file(filename)
        
Widget1ExampleConfiguration = interactive(chooseAndPlotConfigurationAltOutput, filename=widget_choose_filename)

# This function updates the dropdown list when a new file is uploaded
def updateWidget1ExampleConfiguration(*args):
    uploaded_filename = next(iter(Widget1UploadConfiguration.value))
    content = Widget1UploadConfiguration.value[uploaded_filename]['content']
    with open("uploaded/" + uploaded_filename, 'wb') as f: f.write(content)
    mypath="uploaded/"
    found_files=[]
    for f in os.listdir(mypath):
        if (f!="README.md"):
            found_files += [(f , mypath + f)]
    found_files = tuple(found_files)
    #found_files = tuple([(f , mypath + f) for f in os.listdir(mypath)])
    all_files = found_files+examples
    widget_choose_filename.options = all_files
    
Widget1UploadConfiguration.observe(updateWidget1ExampleConfiguration,names='value')

Widget1ExamplesAndUpload = widgets.HBox([Widget1ExampleConfiguration,Widget1UploadConfiguration])
Widget1Text = HTML(markdown.markdown("""
Choose a structure from the examples or upload your own in .cif format [https://en.wikipedia.org/wiki/Crystallographic_Information_File].
"""))
Widget1 = widgets.VBox([Widget1Text,Widget1ExamplesAndUpload,Widget1Out])

In [42]:
#############################
# STEP 2: Run cell2info
#############################
uploaded_filename_refcode = next(iter(Widget1UploadConfiguration.value)).split('/')[-1].split('.')[-1]
cif_content = Widget1UploadConfiguration.value[uploaded_filename]['content']
#cif_content = codecs.decode(widget_choose_filename.value, encoding="utf-8")
with open("uploaded/" + uploaded_filename_refcode +'.cif', 'wb') as f: f.write(cif_content)
refcode = uploaded_filename.split('.')[-2]
info_path = input_path.replace('.cif','.info')
error_path = input_path.replace('.cif','.error')
cif_2_info(input_path, info_path, error_path)


StopIteration: 

In [ ]:
#############################
# STEP 3: Run cell2mol on infofile
#############################
cell = cell2mol(info_path, refcore, os.getcwd(), 3)


DownloadButton(filename='foo.txt', contents=lambda: f'hello {time.time()}', description='download')

In [ ]:
html_acknowledgments = markdown.markdown("""
* Funding from the NCCR MARVEL funded by the SNSF.
""")
acknowledgments=widgets.VBox([
    HTML(html_acknowledgments)
])

html_howtocite = markdown.markdown("""
Please cite the cell2mol paper~!
""")
howtocite=widgets.VBox([
    HTML(html_howtocite)
])

post_children=[acknowledgments,howtocite]
post_accordion = widgets.Accordion(layout=widgets.Layout())
#print(post_accordion.layout.keys)
post_accordion.children = post_children
post_accordion.set_title(0, 'Acknowledgments')
post_accordion.set_title(1, 'How to cite')
post_accordion.selected_index = None
post_accordion

See the project on [GitHub](https://github.com/lcmd-epfl/cell2mol)

cell2mol, Copyright © 2022 LCMD-EPFL